In [19]:

import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
from wordcloud import WordCloud

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB

In [2]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [32]:
cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [33]:
df = pd.read_csv("111.csv")

In [34]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)

    return input_txt  
# remove twitter sentences in @xxx format
df['handles_removed'] = np.vectorize(remove_pattern)(df['Text'], "@[\w]*")
df.head(5)

,Number,Label,Text,handles_removed
0,6.289490e+17,negative,dear @Microsoft the newOoffice for Mac is grea...,"dear the newOoffice for Mac is great and all,..."
1,6.289770e+17,negative,@Microsoft how about you make a system that do...,how about you make a system that doesn't eat ...
2,6.290230e+17,negative,I may be ignorant on this issue but... should ...,I may be ignorant on this issue but... should ...
3,6.291790e+17,negative,"Thanks to @microsoft, I just may be switching ...","Thanks to , I just may be switching over to ."
4,6.291860e+17,neutral,If I make a game as a #windows10 Universal App...,If I make a game as a #windows10 Universal App...


In [35]:
# remove special characters, numbers, punctuations
df['spec_char_removed'] = df['handles_removed'].str.replace("[^a-zA-Z#]", " ")
df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Number,Label,Text,handles_removed,spec_char_removed
0,6.289490e+17,negative,dear @Microsoft the newOoffice for Mac is grea...,"dear the newOoffice for Mac is great and all,...",dear the newOoffice for Mac is great and all ...
1,6.289770e+17,negative,@Microsoft how about you make a system that do...,how about you make a system that doesn't eat ...,how about you make a system that doesn t eat ...
2,6.290230e+17,negative,I may be ignorant on this issue but... should ...,I may be ignorant on this issue but... should ...,I may be ignorant on this issue but should ...
3,6.291790e+17,negative,"Thanks to @microsoft, I just may be switching ...","Thanks to , I just may be switching over to .",Thanks to I just may be switching over to
4,6.291860e+17,neutral,If I make a game as a #windows10 Universal App...,If I make a game as a #windows10 Universal App...,If I make a game as a #windows Universal App...


In [36]:
df['tidy'] = df['spec_char_removed'].apply(lambda x: ' '.join([i for i in x.split(" ") if len(i)>3]))
df[["Label", "Text", "tidy"]].head(5)

,Label,Text,tidy
0,negative,dear @Microsoft the newOoffice for Mac is grea...,dear newOoffice great Lync update
1,negative,@Microsoft how about you make a system that do...,about make system that doesn friggin discs Thi...
2,negative,I may be ignorant on this issue but... should ...,ignorant this issue should celebrate parental ...
3,negative,"Thanks to @microsoft, I just may be switching ...",Thanks just switching over
4,neutral,If I make a game as a #windows10 Universal App...,make game #windows Universal Will #xboxone own...


In [38]:
tokenized_tweet = df['tidy'].apply(lambda x: x.split(" "))
tokenized_tweet

0                 [dear, newOoffice, great, Lync, update]
1       [about, make, system, that, doesn, friggin, di...
2       [ignorant, this, issue, should, celebrate, par...
3                         [Thanks, just, switching, over]
4       [make, game, #windows, Universal, Will, #xboxo...
                              ...                        
5863    [good, know, Punting, MetLife, December, task,...
5864    [everyone, around, metlife, annoying, didnt, r...
5865    [what, giants, niners, fans, would, wanna, sun...
5866    [Anybody, want, ticket, tomorrow, Colombia, Pe...
5867    [Mendez, told, drive, MetLife, Sunday, reactio...
Name: tidy, Length: 5868, dtype: object

In [39]:
labels = df["Label"].to_list()
print(labels)

contents = df["tidy"].to_list()
print(contents)

['negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'positive', 'negative', 'positive', 'neutral', 'negative', 'positive', 'negative', 'posi

In [41]:
# word-> matrix: a[i][j] is word frequency of j under text i
vectorizer = CountVectorizer()

# Count tf-idf weights for each word
transformer = TfidfTransformer()

#fit_transform: tf-idf 
tfidf = transformer.fit_transform(vectorizer.fit_transform(contents)) # fit_transform: convert text into word-frequency matrix


# Get all phrases in bag-of-words model 
word = vectorizer.get_feature_names()
print("Length of words:", len(word))

#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
X = coo_matrix(tfidf, dtype=np.float32).toarray() 
print(X.shape)
print(X[:10])

Length of words: 12591
(5868, 12591)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=1)

# LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)
print("LogisticRegression:")
print('Accuracy:{}'.format(LR.score(X_test, y_test)))
pre = LR.predict(X_test)

print(len(pre), len(y_test))
print(classification_report(y_test, pre))
print("\n")
print(pre)
print(y_test)


LogisticRegression:
Accuracy:0.58603066439523
1761 1761
              precision    recall  f1-score   support

    negative       0.65      0.06      0.10       264
     neutral       0.49      0.35      0.41       591
    positive       0.62      0.89      0.73       906

    accuracy                           0.59      1761
   macro avg       0.59      0.43      0.41      1761
weighted avg       0.58      0.59      0.53      1761



['positive' 'positive' 'neutral' ... 'neutral' 'positive' 'positive']
['positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'neutral', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'neutral', 'negative', 'neut

In [47]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20)
clf.fit(X_train, y_train)
print("RandomForestClassifier:")
print('Accuracy:{}'.format(clf.score(X_test, y_test)))
print("\n")
pre = clf.predict(X_test)
print(len(pre), len(y_test))
print(classification_report(y_test, pre))


RandomForestClassifier:
Accuracy:0.5661555934128336


1761 1761
              precision    recall  f1-score   support

    negative       0.55      0.13      0.21       264
     neutral       0.46      0.32      0.38       591
    positive       0.60      0.85      0.71       906

    accuracy                           0.57      1761
   macro avg       0.54      0.44      0.43      1761
weighted avg       0.55      0.57      0.52      1761

